<a href="https://colab.research.google.com/github/linhtoroi/hihi/blob/main/DS_Test_T12_THUC_HANH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ĐÁNH GIÁ NĂNG LỰC KỸ SƯ KHOA HỌC DỮ LIỆU THÁNG 12 - 2021
#### BÀI THỰC HÀNH
---

Ngày test: 24/12/2021
Thời gian làm bài: 60 phút, từ 10:00 đến 11:00
(+ 5 phút thiết lập môi trường và nộp bài)

Đề thi có 12 câu, tương đương 20 điểm

Quy định:
- Không trao đổi trong suốt quá trình làm bài
- Record màn hình làm bài và share lại cho quanvh8 sau khi kết thúc thời gian test
- Được sử dụng tài liệu và tra cứu internet
- Sau khi hoàn thành, tải notebook, gửi về email: quanvh8@viettel.com.vn

Hướng dẫn làm bài:

* Chạy toàn bộ lệnh trong phần 0. CHUẨN BỊ MÔI TRƯỜNG
* Làm theo hướng dẫn tại các câu và CHỈ SỬA các đoạn code nằm giữa các thẻ:

      # YOUR CODE START
      # / END YOUR CODE

*Lưu ý: hầu hết câu hỏi không có tính tuần tự (không cần hoàn thành câu trước để làm câu sau)*

*Đề thi có một số testcase hỗ trợ DS kiểm tra kết quả, nhưng đây không phải điểm số. Điểm số được giám khảo chấm và gửi lại sau.*



---



## 0. CHUẨN BỊ MÔI TRƯỜNG 
Guide: Chạy toàn bộ các lệnh trong phần này

In [ ]:
!pip install pyspark==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 71.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805913 sha256=dc3809ce4c151e412534e4bd06c79f29bf03b8179300d1738323c761f383b0cc
  Stored in directory: /root/.cache/pip/wheels/23/f6/d3/110e53bd43baeb8d7d38049733d48e39cbecd056f01dba7ee8
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import requests 
import seaborn as sns
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import functions as F

In [ ]:
print(pyspark.__version__)

3.2.0


In [ ]:
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("DS_test") \
    .getOrCreate()

#### DATA
Bộ dữ liệu "Student Performance Data Set", 
Xem thêm bộ dữ liệu này tại: [UCI Student Performance](https://archive.ics.uci.edu/ml/datasets/Student+Performance)

In [ ]:
# Tải dữ liệu về
file_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip'
file_path = "/content/student.zip"

r = requests.get(file_url, stream = True) 

with open( file_path, "wb") as file: 
    for block in r.iter_content(chunk_size = 1024*10): 
        if block: 
            file.write(block)

In [ ]:
!unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [ ]:
# TEST CASE
class TEST():
  def test_1(df_math, df_por):
    tcs = lambda df: [len(df.columns), df.count()]
    if tcs(df_math) == [33, 395]: print('df_math: PASS') 
    else : print('df_math: FAIL')
    if tcs(df_por) == [33, 649]: print('df_por: PASS') 
    else : print('df_por: FAIL')

  def test_2(sql_text):
    check = spark.sql(sql_text).toPandas().sort_values('sex')['student_num'].values
    if [check.sum(), check[0]/check[1] ] == [21, 1.1]: print('PASS')
    else: print('FAIL')

## 1. BIG DATA MANIPULATION AND ANALYSIS

Câu 1. [2p, 1đ] Đọc dữ liệu từ file

Sau khi tải về và giải nén, bộ dữ liệu gồm 2 file:
* student-mat.csv : dữ liệu điểm môn toán
* student-por.csv : dữ liệu điểm môn tiếng Bồ Đào Nha

YC: Đọc 2 file này vào các pyspark dataframe tương ứng df_math, df_por

In [ ]:
df = spark.read.option("header",'True').option('delimiter', ';').csv(file_path_math)

In [ ]:
df.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 17|    

In [ ]:
file_path_math = "/content/student-mat.csv"
file_path_por = "/content/student-por.csv"

df_math = spark.read.option("header",'True').option("inferSchema",True).option('delimiter', ';').csv(file_path_math)
# (
#     # YOUR CODE START 
    
    
#     # / END YOUR CODE
# )

df_por = spark.read.option("header",'True').option("inferSchema",True).option('delimiter', ';').csv(file_path_por)
# (
#     # YOUR CODE START 
    
    
#     # / END YOUR CODE
# )

In [ ]:
TEST.test_1(df_math, df_por)

df_math: PASS
df_por: PASS


### 1.1. SPARK SQL
Sử dụng SQL để thực hiện các ý sau với bảng tbl_math, và tbl_por

In [ ]:
df_math.createOrReplaceTempView('tbl_math')
df_math.createOrReplaceTempView('tbl_por')

Câu 2.[ 2p, 1 đ]
Xác định nhóm ngành nghề của mẹ (cột Mjob) nào tương ứng với điểm số cả năm (cột g3) của người con cao nhất

In [ ]:
df_math.show(5)

+------+---+---+-------+-------+-------+----+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 17|      U|    GT3|      T|

In [ ]:
df_por.show(5)

+------+---+---+-------+-------+-------+----+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       4|  0| 11| 11|
|    GP|  F| 17|      U|    GT3|      T|

In [ ]:
df_math.groupby('Mjob').agg(F.mean('G3').alias('mean_G3')).orderBy(F.col('mean_G3').desc()).show()

+--------+------------------+
|    Mjob|           mean_G3|
+--------+------------------+
|  health|12.147058823529411|
| teacher|11.051724137931034|
|services|11.019417475728156|
|   other| 9.822695035460994|
| at_home| 9.152542372881356|
+--------+------------------+



In [ ]:
sql_text = (
# YOUR CODE START 
'''
SELECT Mjob FROM ((SELECT Mjob, G3 FROM tbl_math) UNION (SELECT Mjob, G3 FROM tbl_por)) GROUP BY Mjob ORDER BY MEAN(G3) DESC LIMIT 1
'''
# / END YOUR CODE
)
spark.sql(sql_text).toPandas()

,Mjob
0,health


Câu 3. [2p, 2đ] Xác định hệ số tương quan Pearson của điểm số học kỳ 1 (cột G1) và điểm số học kỳ 2 (cột G2)

In [ ]:
df_math.show(5)

+------+---+---+-------+-------+-------+----+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 17|      U|    GT3|      T|

In [ ]:
sql_text = (
# YOUR CODE START 
'''
SELECT CORR(G1, G2) from tbl_math
'''
# / END YOUR CODE
)
spark.sql(sql_text).toPandas() 

,"corr(G1, G2)"
0,0.852118


Câu 4.[ 10p, 2đ]
Sở Giáo dục dự kiến cấp học bổng cho các học sinh dựa trên kết quả học tập cả năm môn toán (bảng tbl_math), sao cho:
* Các học sinh có kết quả học tập cả năm (cột G3) cao hơn thì ưu tiên lựa chọn.
* Tỷ trọng số lượng học sinh theo từng gới tính (cột sex) được nhận học bổng là như nhau: số học sinh nam nhận học bổng trên tổng số học sinh nam bằng số học sinh nữ nhận học bổng trên tổng số nữ.
* Tổng số thời gian vắng mặt (cột absences) của các học sinh được nhận học bổng không quá 5% tổng thời gian vắng mặt của tất cả học sinh.

YC: Viết lệnh SQL truy xuất số lượng học sinh được nhận học bổng của mỗi giới tính. Bảng đầu ra có các cột: {sex:string, student_num:int}

In [ ]:
sql_text = (
# YOUR CODE START 
'''
WITH table_pct AS(
  SELECT  *, ROW_NUMBER() OVER (PARTITION BY sex ORDER BY g3 DESC)
            / COUNT(1) OVER (PARTITION BY sex) as pct
  FROM tbl_math)
SELECT sex, COUNT(*) AS student_num FROM 
(
SELECT *, SUM(absences) OVER(ORDER BY pct) /SUM(absences) OVER() AS total
  FROM table_pct
)
WHERE total < 0.05
GROUP BY sex
'''
# / END YOUR CODE
)
spark.sql(sql_text).toPandas()

,sex,student_num
0,F,11
1,M,10


In [ ]:
TEST.test_2(sql_text)

PASS


### 1.2. PYSPARK
Sử dụng pyspark và các dataframe df_math, df_por trả lời các câu hỏi sau

Câu 5. [2p, 1đ] Xác định số lượng học sinh vừa học môn toán, vừa học tiếng Bồ Đào Nha. Biết rằng, mỗi sinh viên được đặc trưng bởi một tổ hợp các cột: "school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"

In [ ]:
# YOUR CODE START
keys = ["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"]
df_math.join(df_por, on=keys).count()
# / END YOUR CODE

382

Câu 6. [2p, 1đ]
Xác định hệ số tương quan Pearson giữa điểm số cả năm môn toán và điểm số cả năm môn tiếng Bồ Đào Nha của các học sinh đều học 2 môn này


In [ ]:
# YOUR CODE START
df1_a = df_math.select([F.col(x).alias(x + '_df_math') if x not in keys else x for x in df_math.columns])
df2_a = df_por.select([F.col(x).alias(x + '_df_por') if x not in keys else x for x in df_por.columns])

df1_a.join(df2_a, on=keys).stat.corr('G3_df_math', 'G3_df_por')
# / END YOUR CODE

0.4803493619410261

Câu 7. [5p. 2đ] Xác định hệ số tương quan Kendall tau của điểm số học kỳ 2 học kỳ (G1 và G2) giữa môn toán và môn tiếng Bồ Đào Nha của từng học sinh (cho các học sinh học cả 2 môn này). Biết rằng pyspark không hỗ trợ , nhưng scipy có hỗ trợ tính Kendall tau tại: scipy.stats.kendalltau.

In [ ]:
from scipy.stats import kendalltau

# YOUR CODE START 
df_merge = df1_a.join(df2_a, on=keys)

@F.pandas_udf("float")
def func(x: pd.Series, y: pd.Series)-> float:
  return kendalltau(x, y).correlation

df_merge.select(func(df_merge['G3_df_math'], df_merge['G3_df_por'])).show()
# / END YOUR CODE

+---------------------------+
|func(G3_df_math, G3_df_por)|
+---------------------------+
|                  0.4188197|
+---------------------------+



In [ ]:
kendalltau(df_merge.select('G3_df_math').rdd.flatMap(lambda x: x).collect(), df_merge.select('G3_df_por').rdd.flatMap(lambda x: x).collect()).correlation

0.4188196925244088

## 2. MODELING

In [ ]:
pddf_por = pd.read_csv(file_path_por, sep=';')
pddf_por.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


Câu 8. [2p, 1đ] Sử dụng pandas dataframe pddf_por, chia ngẫu nhiên thành 2 tập train, test với tỷ lệ 7:3

In [ ]:
# YOUR CODE START 


# / END YOUR CODE

Câu 9. [5p, 3đ] Tiền xử lý dữ liệu

* Với các nhãn G1, G2, G3: giữ nguyên
* Với các features (các cột còn lại): 
  * Với cột dạng số: áp dụng StandardScaler
  * Với cột dạng yes/no: chuyển về dạng 0/1
  * Với cột dạng biến danh mục: sử dụng onehot-encoder

Fit-transform theo luồng trên cho tập train, transform cho tập test để tạo bộ feature

In [ ]:
# YOUR CODE START 

# / END YOUR CODE

Câu 10. [9p, 2đ] Chọn một mô hình và huấn luyện dự đoán đồng thời 3 nhãn G1, G2, G3. Đánh giá hiệu quả model trên tập test với chỉ tiêu mean absolute error (MAE). Sao cho trung bình MAE 3 nhãn trên tập test <=1.98. 

Tính giá trị MAE từng nhãn khi đó.

In [ ]:
# YOUR CODE START 

# / END YOUR CODE

Câu 11. [9p, 2đ] Xây dựng một mạng học sâu với tối thiểu 2 hidden layers với các tham số phù hợp để dự đoán các nhãn G1, G2, G3. Sao cho trung bình MAE 3 nhãn trên tập test <=1.98. 

Tính giá trị MAE từng nhãn khi đó.

In [ ]:
# YOUR CODE START 

# / END YOUR CODE

Câu 12. [10p, 2đ] Áp dụng transfer learning, hãy sử dụng mạng vừa tạo để xây dựng mô hình dự đoán điểm môn toán.
1. Tiền xử lý dữ liệu tương tự như với bộ pddf_por
2. Đóng băng 2 layer đầu của model đã huấn luyện trên bộ Tiếng Bồ Đào Nha, bổ sung thêm ít nhất 2 layer phía sau tạo thành mô hình dự đoán 3 đầu ra môn Toán.
3. Huấn luyện mô hình dự đoán 3 điểm số môn toán (G1, G2, G3) và đánh giá bằng chỉ tiêu mean square error (MAE).Sao cho trung bình MAE 3 nhãn trên tập test <= 3.0
4. Khi đó, MAE của từng nhãn G1, G2, G3 môn toán là bao nhiêu?

In [ ]:
pddf_math = pd.read_csv(file_path_math, sep=';')
# YOUR CODE START 

# / END YOUR CODE